In [1]:
import pandas as pd

#Etape 1 : Importer le dataset pour créer un dataframe tout en retirant les doublons
df = pd.read_csv("./Dataset/reseau_en_arbre.csv").drop_duplicates()

#Création d'un Dataframe ne contenant uniquement les informations dont infra_type = a_remplacer
network_df = df[df["infra_type"] == "a_remplacer"]
network_df.head()

,id_batiment,nb_maisons,infra_id,infra_type,longueur
10,E000003,1,P007113,a_remplacer,12.193509
11,E000003,1,P007984,a_remplacer,30.057598
12,E000003,1,P007823,a_remplacer,12.290283
465,E000067,1,P006193,a_remplacer,12.186388
466,E000067,1,P002851,a_remplacer,10.803202


A ce niveau, on importe les classes batiment et infra qui nous permettront de modéliser nos données selon notre dataset

In [2]:
from packages.infra import *
from packages.batiment import *

In [3]:
# Création de fonction modeling_data() dont le rôle est de créer des modèles de batiments et d'infrastructure
    # Cette fonction prend en paramètre un dataset 
    # Pour cela nous créerons un dictionnaire d'infrastructure contenant les données de notre jeu de données
    # Ensuite, créerons une liste de batiments avec chaque batiment reliés à ses infrastructures

def modeling_data(network_df):

    dict_infras = {}

    # Création d'un dataframe groupé par infra_id
    infra_subdfs = network_df.groupby(by="infra_id")

    #Création du dictionnaire d'infrastructures
    for id_infra, infra_subdf in infra_subdfs:
        length = infra_subdf["longueur"].iloc[0]
        nb_houses = infra_subdf["nb_maisons"].sum()
        infra_type = infra_subdf["infra_type"].iloc[0]
        dict_infras[id_infra] = Infra(id_infra, nb_houses, infra_type, length)


    list_of_buildings = []

    # Création d'un dataframe groupé par batiment_id
    bat_subdfs = network_df.groupby(by="id_batiment")

    #Création d'une liste de batiments
    for id_bat, bat_subdf in bat_subdfs:
        list_infras_building = [dict_infras[id_infra] for id_infra in bat_subdf["infra_id"].values ]
        batiment = Batiment(id_bat, list_infras_building)
        list_of_buildings.append(batiment)

    return  list_of_buildings
    
list_buildings = modeling_data(network_df)



Passons maintenant à la création de la fonction logique de notre projet à savoir la fonction connection_planning(). Cette fonction permettra de définir un planning de raccordement suivant le critère prioritaire de la commune. 
Rappelons que les batiments les plus prioritaires sont ceux qui ont la **plus faible difficulté de raccordement**

In [4]:
# Création de fonction coeur de notre projet : connection_planning()
    #Cette fonction a pour rôle de parcourir la list de batiment de récupérer le batiment ayant la plus petite difficulté 
    # et l'insérer dans une liste de batiments classés par ordre de prioritéé. 
    # Elle assure aussi le changement d'état d'une infrastructure soit de l'état "a_remplacer" à "infra_intacte"



def connection_planning( list_buildings):
    
    list_buildings_by_priority = []

    while list_buildings:
        # On récupère le batiment le moins difficile à raccorder
        priority_building = min(list_buildings)
        

        # On l'ajoute dans la liste des batiments classés par priorité
        list_buildings_by_priority.append(priority_building)
        
        #On parcourt sa liste d'infrastructure pour changer leur état
        for infra in priority_building.list_infras:
            infra.repair_infra()
        list_buildings.remove(priority_building)

    return list_buildings_by_priority

        
        #Est ce que la modification de l'état des infrastructures d'un batiment modifie également l'état des infras des batiments ayant les mêmes infra ????

list_buildings = modeling_data(network_df)
bat = connection_planning(list_buildings)

###########################         DEBUGAGE    ###################
# print(bat[0])
# print(bat[4].list_infras[1].infra_id)
# print(bat[4].list_infras[2].infra_type)
# print("_"*30)
# print(bat[5])
# print(bat[5].list_infras[1].infra_id)
# print(bat[5].list_infras[2].infra_type)





    


La fonction ci-dessus permet d'exécuter notre projet à partir du fichier main.py

In [9]:
if __name__ == "__main__":
    list_buildings = modeling_data(network_df)
    list_of_buildings_sorted = connection_planning(list_buildings)

    #Ici, on veut récupérer et stocker dans les listes buildings_ids et positions_list respectivement un batiment et sa position. 
    positions_list, buildings_ids = [], []
    for index, building in enumerate(list_of_buildings_sorted):
        positions_list.append(index)
        buildings_ids.append(building.id_building)

    # Puis on créer un dictionnaire qui contient ces 02 listes puis, on le transforme en un Dataframe
    pd.DataFrame({"priority_order": positions_list,
                  "building_id": buildings_ids}).to_excel("./connection_planning.xlsx", index=False)

    
    
